In [1]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timedelta

# 設置MySQL資料庫連接
db_user = 'root'
db_password = '19970730'
db_host = '127.0.0.1'
db_name = 'sql_stock'

# 創建資料庫連接引擎
engine = create_engine(f'mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}')

# 設置日期範圍
#today = datetime(2024, 5, 25)
today = datetime.now().date()
one_year_ago = today - timedelta(days=1000)

# 格式化日期
today_str = today.strftime('%Y%m%d')
one_year_ago_str = one_year_ago.strftime('%Y%m%d')
one_year_ago_str = '20210601'

# SQL 查詢 加權指數
query_twse = f"""
SELECT *
FROM daily_twse
WHERE date >= '{one_year_ago_str}' AND date <= '{today_str}'
"""

# SQL 查詢 個股
query_stock = f"""
SELECT *
FROM daily_price
WHERE date >= '{one_year_ago_str}' AND date <= '{today_str}'
"""

# SQL 查詢 細產業
query_sub_category = f"""
SELECT *
FROM sub_category
"""

# 使用 pd.read_sql 來執行查詢並讀取數據到 DataFrame
twse_df_raw = pd.read_sql(query_twse, engine)

stock_df_raw = pd.read_sql(query_stock, engine)

sub_category_list = pd.read_sql(query_sub_category, engine)



In [2]:
from fubon_neo.sdk import FubonSDK, FutOptOrder
from fubon_neo.constant import TimeInForce, FutOptOrderType, FutOptPriceType, FutOptMarketType, BSAction

sdk = FubonSDK()

accounts = sdk.login("D122948227", "mulder304028", "C:\\Users\\USER\\OneDrive\\桌面\\佛爾德\\stock\\CAFubon\\D122948227\\D122948227.pfx","mulder304028")

print(accounts) #若有多帳號，則回傳多個

#建立委託單內容
order = FutOptOrder(
    buy_sell = BSAction.Buy,
    symbol = "TMFJ4",  #是這個吧 微台指 十月
    price = "22000",
    lot =  1,
    market_type = FutOptMarketType.Future,
    price_type = FutOptPriceType.Limit,
    time_in_force= TimeInForce.ROD,
    order_type = FutOptOrderType.Auto,
    user_def = "From Python" # optional field
) 


sdk.futopt.place_order(accounts.data[0], order)  #下單委託

Result {
  is_success: True,
  message: None,
  data: [Account {
    name: "方品謙",
    branch_no: "15000",
    account: "277547",
    account_type: "futopt",
}]
}


In [23]:
orderResults = sdk.futopt.get_order_results(accounts.data[0])
print(orderResults)

Result {
  is_success: True,
  message: None,
  data: [FutOptOrderResult {
    function_type: None,
    date: "2024/09/23",
    seq_no: "00116202331",
    branch_no: "15000",
    account: "277547",
    order_no: None,
    asset_type: 1,
    market: "TAIMEX",
    market_type: Future,
    symbol: "FITM",
    unit: None,
    currency: "TWD",
    expiry_date: "202410",
    strike_price: None,
    call_put: None,
    buy_sell: Buy,
    symbol_leg2: None,
    expiry_date_leg2: None,
    strike_price_leg2: None,
    call_put_leg2: None,
    buy_sell_leg2: None,
    price_type: Limit,
    price: 22000,
    after_price: 22000,
    lot: 1,
    after_lot: 0,
    time_in_force: ROD,
    order_type: UnDefined,
    status: 90,
    is_pre_order: false,
    after_price_type: None,
    filled_lot: 0,
    filled_money: 0,
    before_lot: None,
    before_price: None,
    user_def: "From Pytho",
    last_time: "11:39:59",
    details: None,
    error_message: "可用保證金不足(後檯)[8481300]",
}]
}


In [26]:
#建立委託單內容
order = FutOptOrder(
    buy_sell = BSAction.Sell,
    symbol = "TMFJ4",
    price = "23000",
    lot =  1,
    market_type = FutOptMarketType.Future,
    price_type = FutOptPriceType.Limit,
    time_in_force= TimeInForce.ROD,
    order_type = FutOptOrderType.Auto,
    user_def = "From Python" # optional field
) 


sdk.futopt.place_order(accounts.data[0], order)  #下單委託

Result {
  is_success: True,
  message: None,
  data: FutOptOrderResult {
    function_type: 0,
    date: "2024/09/23",
    seq_no: "00116208032",
    branch_no: "15000",
    account: "277547",
    order_no: "s063p",
    asset_type: 1,
    market: "TAIMEX",
    market_type: Future,
    symbol: "FITM",
    unit: None,
    currency: "TWD",
    expiry_date: "202410",
    strike_price: None,
    call_put: None,
    buy_sell: Sell,
    symbol_leg2: None,
    expiry_date_leg2: None,
    strike_price_leg2: None,
    call_put_leg2: None,
    buy_sell_leg2: None,
    price_type: Limit,
    price: 23000,
    after_price: 23000,
    lot: 1,
    after_lot: 1,
    time_in_force: ROD,
    order_type: Auto,
    status: 10,
    is_pre_order: false,
    after_price_type: Limit,
    filled_lot: 0,
    filled_money: 0,
    before_lot: 0,
    before_price: 23000,
    user_def: "From Pytho",
    last_time: "13:38:31",
    details: None,
    error_message: None,
}
}

In [24]:
# 權益數查詢
equity = sdk.futopt_accounting.query_margin_equity(accounts.data[0])
print(equity.data)

[Equity({
  "date": "2024/09/23",
  "branch_no": "15000",
  "account": "277547",
  "currency": "NTD",
  "yesterday_balance": 23070.0,
  "today_balance": 23070.0,
  "initial_margin": 14600.0,
  "maintenance_margin": 11200.0,
  "clearing_margin": 10800.0,
  "today_equity": 28080.0,
  "today_deposit": 0.0,
  "today_withdrawal": 0.0,
  "today_trading_fee": 0.0,
  "today_trading_tax": 0.0,
  "receivable_premium": 0.0,
  "payable_premium": 0.0,
  "excess_margin": 13480.0,
  "available_margin": 12530.0,
  "disgorgement": 0.0,
  "opt_pnl": 0.0,
  "opt_value": 0.0,
  "opt_long_value": 0.0,
  "opt_short_value": 0.0,
  "fut_realized_pnl": 0.0,
  "fut_unrealized_pnl": 5010.0,
  "buy_lot": 1.0,
  "sell_lot": 0.0
}), Equity({
  "date": "2024/09/23",
  "branch_no": "15000",
  "account": "277547",
  "currency": "TWD",
  "yesterday_balance": 23070.0,
  "today_balance": 23070.0,
  "initial_margin": 14600.0,
  "maintenance_margin": 11200.0,
  "clearing_margin": 10800.0,
  "today_equity": 28080.0,
  "toda